## Set up

In [1]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from gensim.models import Phrases
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
from gensim.models import LdaModel

In [2]:
data = pd.read_csv("data/cleaned_data2.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_16694/2170368664.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data2.csv")


## Preprocessing

Tokenizer

In [3]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
data["tokenized"] = data["cleaned_text"].apply(lambda x: tokenizer.tokenize(x))

Stop words

In [4]:
#Custom defined list
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "wasnt", "yeah", "okay", "lol"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "op"]

topic_specific = ["self", "improvement", "selfimprovement", "rselfimprovement"]

stop_words = english + time + reddit + topic_specific

In [5]:
#Remove stopwords
def remove_stopwords_numbers(tokens):
    """Removes stopwords and tokens that contain any digits"""
    return [token for token in tokens if token not in stop_words and not re.search(r'\d', token)]

# Apply function to column
data["filtered_tokens"] = data["tokenized"].apply(remove_stopwords_numbers)

In [6]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes a list of tokens"""
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply function to column
data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

In [7]:
#https://www.h2kinfosys.com/blog/part-of-speech-tagging-chunking-with-nltk/
def get_wordnet_pos(tag):
    """Convert POS tag to a format recognized by WordNetLemmatizer"""
    if tag.startswith('J'): #NLTK tags adjectives as JJ (normal), 
                            #JJR (comparative) and JJS (superlative)
        return wordnet.ADJ  #and then it can be mapped to the wordnet database
                              #to get the lemma
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_tokens(tokens):
    """Lemmatizes tokens with POS tagging"""
    tagged_tokens = pos_tag(tokens)  
    return [lemmatizer.lemmatize(word, pos) for word, tag in tagged_tokens
            if (pos := get_wordnet_pos(tag))]

data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

In [8]:
data[["lemmatized"]]

,lemmatized
0,"[appointment, dentist, last, reminder, email, ..."
1,"[site, several, month, release, beta, form, ra..."
2,"[everyone, recently, change, overall, enough, ..."
3,"[body, dysmorphia, disorder, social, anixety, ..."
4,"[point, content, never, process, stop, speak, ..."
...,...
507604,"[prepared, someday, gonna, matter, back, fact,..."
507605,"[grow, external, validation, actual, happy, me..."
507606,"[convenient, reason, unhappy, factor, external..."
507607,"[trust, old, superpower, age, awkward, everyon..."


Bigrams

In [9]:
# Train bigram model on the lemmatized column
bigram = Phrases(data["lemmatized"], min_count=20)
bigram_phraser = Phraser(bigram)  # Optimizes since it is a lighter-weight
                                   #version of Phrases

def add_bigrams(tokens):
    """Adds bigrams to a list of tokens if they appear frequently enough."""
    bigram_tokens = bigram_phraser[tokens]
    return tokens + [token for token in bigram_tokens if '_' in token]

data["ngrams"] = data["lemmatized"].apply(add_bigrams)

Filtering of too rare of too common words

In [10]:
# Convert df column to a list of lists for later steps
docs = data["ngrams"].tolist() 

# Create a dictionary from the tokenized documents
dictionary = Dictionary(docs)

# Filter out words that appear in fewer than 20 documents or 
#  more than 80% of documents, and keep the 1500 most frequent
dictionary.filter_extremes(no_below=20, no_above=0.8, keep_n=1500)


Vectorize

In [11]:
# Create bag of words representation
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [42]:
corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1)]

TF IDF Filtering

In [18]:
from gensim.models import TfidfModel
temp = dictionary[0]  # Load the dictionary in memory 
tfidf = TfidfModel(corpus, id2word=dictionary.id2token,  dictionary=dictionary)
tfidf_corpus = tfidf[corpus] #convert to weighted corpus

In [50]:
tfidf_threshold = 0.10 

#Create new list to store filtered corpus by TF-IDF scores
filtered_corpus = []

for doc in corpus:
    tfidf_doc = tfidf[doc]  # Get TF-IDF scores for the document
    tfidf_dict = dict(tfidf_doc)  # Convert to dictionary {token_id: tfidf_score}

    # Keep words from original BOW if their TF-IDF score is above the threshold
    filtered_doc = [(token_id, count) for token_id, count in doc if tfidf_dict.get(token_id, 0) > tfidf_threshold]
    
    filtered_corpus.append(filtered_doc)

## Training

In [51]:
num_topics = 10
chunksize = 2000 #to speed up training
passes = 10
iterations = 100
eval_every = None
minimum_probability = 0.10  
alpha = 0.2

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

model = LdaModel(
    corpus=filtered_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

In [55]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -2.4991.
[([(0.03218448, 'even'),
   (0.03017872, 'bad'),
   (0.028814925, 'hard'),
   (0.02834778, 'well'),
   (0.019666387, 'everything'),
   (0.019423438, 'back'),
   (0.015077725, 'change'),
   (0.014739713, 'best'),
   (0.012448599, 'try'),
   (0.012404177, 'good'),
   (0.012116911, 'stop'),
   (0.010669303, 'sometimes'),
   (0.010175822, 'first'),
   (0.009396984, 'need'),
   (0.008899387, 'easy'),
   (0.00876957, 'little'),
   (0.008472593, 'matter'),
   (0.008214693, 'actually'),
   (0.008124072, 'shit'),
   (0.008095185, 'time')],
  -2.0278150915057602),
 ([(0.044777915, 'goal'),
   (0.032973014, 'book'),
   (0.0299162, 'step'),
   (0.027918285, 'small'),
   (0.027774546, 'start'),
   (0.0197748, 'habit'),
   (0.016458094, 'great'),
   (0.015050282, 'term'),
   (0.014921134, 'first'),
   (0.013342873, 'read'),
   (0.012248663, 'change'),
   (0.011552645, 'progress'),
   (0.010647659, 'write'),
   (0.010219643, 'long'),
   (0.010178938, 'plan'),
   (0.0

In [53]:
from gensim.models.coherencemodel import CoherenceModel

In [54]:
cm = CoherenceModel(model=model, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary)
cm.get_coherence()  


0.500950930647863